In [408]:
import pandas as pd
import numpy as np
import nltk
import re

nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords 
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abdullrahmannawasrah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/abdullrahmannawasrah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [82]:
# Create filenames 

filenames = ['arxiv_annotate1_13_1.txt', 'arxiv_annotate1_13_2.txt', 'arxiv_annotate1_13_3.txt','arxiv_annotate2_66_1.txt', 'arxiv_annotate2_66_2.txt', 
  'arxiv_annotate3_80_3.txt', 'arxiv_annotate4_168_1.txt', 'arxiv_annotate4_168_2.txt','arxiv_annotate4_168_3.txt', 'arxiv_annotate5_240_1.txt']

  


with open('file.txt', 'w') as outfile: 

  

    

    for names in filenames: 

  

        

        with open(names) as infile: 

  

           

            outfile.write(infile.read()) 

  

    

        outfile.write("\n")

Open text then covert text into csv file

In [87]:
doc=open("file.txt", "r")
doc=(doc.read())
corpus = pd.read_csv("file.txt", delimiter = '/')
corpus.to_csv('file.txt',index = None)

Load csv file

In [279]:
corpus=pd.read_csv("file.csv", sep='\t',names=["labels","Sentences"])
corpus


,labels,Sentences
0,### abstract ###,NaN
1,MISC,although the internet as level topology has be...
2,MISC,an as node can represent a wide variety of o...
3,AIMX,in this paper we introduce a radically new ap...
4,OWNX,we successfully classify NUMBER NUMBER per...
...,...,...
330,OWNX,fig is just an example
331,MISC,however a similar situation often occurs in c...
332,MISC,clustering algorithms in most cases give alm...
333,MISC,thus a better clustering assignment can be co...


Clean data like remove stopwords and  punctuations. Also, remove ### abstract ### and introduction

In [280]:
corpus['Sentences']=corpus['Sentences'].str.lower()
corpus

,labels,Sentences
0,### abstract ###,NaN
1,MISC,although the internet as level topology has be...
2,MISC,an as node can represent a wide variety of o...
3,AIMX,in this paper we introduce a radically new ap...
4,OWNX,we successfully classify number number per...
...,...,...
330,OWNX,fig is just an example
331,MISC,however a similar situation often occurs in c...
332,MISC,clustering algorithms in most cases give alm...
333,MISC,thus a better clustering assignment can be co...


In [281]:
corpus=corpus.dropna()
corpus=corpus.dropna(axis=0)


In [282]:
corpus['Sentences'] = corpus['Sentences'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))
corpus

,labels,Sentences
1,MISC,although internet level topology extensively s...
2,MISC,node represent wide variety organizations e g ...
3,AIMX,paper introduce radically new approach based m...
4,OWNX,successfully classify number number percent as...
5,OWNX,release community level topology dataset augme...
...,...,...
330,OWNX,fig example
331,MISC,however similar situation often occurs clustering
332,MISC,clustering algorithms cases give almost global...
333,MISC,thus better clustering assignment constructed ...


In [283]:
corpus['Sentences']= corpus['Sentences'].str.replace(r'[^\w\s]+', ' ',regex=True)
corpus

,labels,Sentences
1,MISC,although internet level topology extensively s...
2,MISC,node represent wide variety organizations e g ...
3,AIMX,paper introduce radically new approach based m...
4,OWNX,successfully classify number number percent as...
5,OWNX,release community level topology dataset augme...
...,...,...
330,OWNX,fig example
331,MISC,however similar situation often occurs clustering
332,MISC,clustering algorithms cases give almost global...
333,MISC,thus better clustering assignment constructed ...


In [284]:
corpus['Sentences']= corpus['Sentences'].str.replace(r'\n',  '',regex=True)
corpus

,labels,Sentences
1,MISC,although internet level topology extensively s...
2,MISC,node represent wide variety organizations e g ...
3,AIMX,paper introduce radically new approach based m...
4,OWNX,successfully classify number number percent as...
5,OWNX,release community level topology dataset augme...
...,...,...
330,OWNX,fig example
331,MISC,however similar situation often occurs clustering
332,MISC,clustering algorithms cases give almost global...
333,MISC,thus better clustering assignment constructed ...


In [285]:
corpus['Sentences']= corpus['Sentences'].str.replace(r'\t',  ' ',regex=True)
corpus

,labels,Sentences
1,MISC,although internet level topology extensively s...
2,MISC,node represent wide variety organizations e g ...
3,AIMX,paper introduce radically new approach based m...
4,OWNX,successfully classify number number percent as...
5,OWNX,release community level topology dataset augme...
...,...,...
330,OWNX,fig example
331,MISC,however similar situation often occurs clustering
332,MISC,clustering algorithms cases give almost global...
333,MISC,thus better clustering assignment constructed ...


In [294]:
corpus['labels'].value_counts()

MISC     65
OWNX     58
CONT     27
AIMX     12
BASE      7
OWNX      1
Name: labels, dtype: int64

In [306]:
sents = corpus['Sentences']
Labels = corpus['labels']

In [377]:
 ##apply TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer

Tfidf = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
Tfidf_ = Tfidf.fit_transform(sents)
Tfidf_ = Tfidf_.toarray()

voc = Tfidf.get_feature_names()
pd.DataFrame(np.round(Tfidf_, 2), columns=voc)

,able,abstract,access,account,accuracy,accurate,accurately,achieves,across,active,...,wide,widely,without,work,working,works,years,yielding,yields,zero
0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.31,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.18,0.0,0.0,0.0,0.2,0.0,0.00,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.23,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
166,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
167,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
168,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [329]:
corpus.dtypes

labels       object
Sentences    object
dtype: object

In [339]:
MLB=MultiLabelBinarizer()

In [373]:
Labels=MLB.fit_transform(corpus["labels"])

In [341]:
# Splitting the data
Labels_train, Labels_test, sents_train, sents_test = train_test_split(Labels, sents, test_size = .4, random_state = 0)

In [342]:
# Use linear support vector Classifier
lsvc = LinearSVC()
# train the model
lsvc.fit(Labels_train, sents_train)

LinearSVC()

In [351]:
print(lsvc.score(Labels_train, sents_train)) 

0.08823529411764706


First, we merge the texts into one text then chnage text into csv file. Secondly,I removed stopwords,punctuations and abstract. Thirdly, I vectorized the Sentences by tfidf. Make model to classify.

In [354]:
# Use RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
# train the model
rfc.fit(Labels_train, sents_train)
print(rfc.score(Labels_train, sents_train))  

0.08823529411764706


In [427]:
pred=rfc.predict(Labels_test)


In [428]:
matrix = classification_report(sents_test,pred)
print('Classification report : \n',matrix)

Classification report : 
                                                                                                                                                                                                                        precision    recall  f1-score   support

                                                                                                                              algorithmic stability used effectively past derive tight generalization bounds citation       0.00      0.00      0.00       1.0
                                         algorithms support vector regression svr citation used context time series prediction independent identically distributed assumption hold good experimental results citation       0.00      0.00      0.00       1.0
 also illustrate application mixing generalization bounds general classes learning algorithms including support vector regression svr citation kernel ridge regression citation support vector machines svms cit

/Applications/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Applications/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Applications/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Applications/anaconda3/lib/python3.